In [13]:
from scipy import ndimage
import cv2
import numpy as np
import os

# сделать изображение документа шумным и не слишком белым
def preprocess_doc_image(image: cv2.Mat):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (31, 31), 0)

    paper_texture_image = cv2.absdiff(gray_image, blurred_image)
    paper_texture_mask = cv2.cvtColor(paper_texture_image, cv2.COLOR_GRAY2BGR)

    output_image = cv2.addWeighted(image, 0.8, paper_texture_mask, 0.2, 0)

    return output_image

def put_image_on_background(img: cv2.Mat, back):
    x,y = 0,0 
    if (back.shape[0] < back.shape[1]): 
        back = cv2.rotate(back, cv2.ROTATE_90_CLOCKWISE)

    scale = min(back.shape[1] / 2 / img.shape[1], back.shape[0] / 2 / img.shape[0])
    x_start = np.random.randint(0, back.shape[0] // 3)
    y_start = np.random.randint(0, back.shape[1] // 3)
    new_img = cv2.resize(img, list(map(int, (scale * img.shape[1], scale * img.shape[0]))))
    back[x_start:x_start+new_img.shape[0],y_start:y_start+new_img.shape[1]] = new_img

    return back

def project_image(img: cv2.Mat):
    points_src = np.float32([[0, 0], [img.shape[1], 0], [0, img.shape[0]], [img.shape[1], img.shape[0]]])

    points_dst = np.float32([[np.random.randint(0, img.shape[1] // 10),np.random.randint(0, img.shape[0] // 10)],
                         [np.random.randint(img.shape[1] // 10 * 9,img.shape[1] - 1),np.random.randint(0, img.shape[0] // 10)],
                         [np.random.randint(0, img.shape[1] // 10),np.random.randint(img.shape[0] // 10 * 9, img.shape[0] - 1)],
                         [np.random.randint(img.shape[1] // 10 * 9,img.shape[1] - 1),np.random.randint(img.shape[0] // 10 * 9, img.shape[0] - 1)]])
    M = cv2.getPerspectiveTransform(points_src, points_dst)

    perspective_image = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))

    points_dst = points_dst.astype(np.int32)
    top_x = max(points_dst[0][0], points_dst[2][0])
    bottom_x = min(points_dst[1][0], points_dst[3][0])
    top_y = max(points_dst[0][1], points_dst[1][1])
    bottom_y = min(points_dst[2][1], points_dst[3][1])
    return perspective_image[top_y:bottom_y:,top_x:bottom_x]


def crop_image(image, angle):
    h, w = image.shape[:2]
    tan_a = abs(np.tan(angle * np.pi / 180))
    b = int(tan_a / (1 - tan_a ** 2) * (h - w * tan_a))
    d = int(tan_a / (1 - tan_a ** 2) * (w - h * tan_a))
    return image[d:h - d, b:w - b]

def rotate_bound(image, angle):
    result = ndimage.rotate(image, angle)
    return crop_image(result, angle)

def add_light(img):
    x, y = np.random.randint(0, img.shape[1]), np.random.randint(0, img.shape[0])
    radius = 3000

    mask = np.zeros_like(img[:,:,0]).astype(np.float32)
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            dist = np.sqrt((i-y)**2 + (j-x)**2)
            if dist < radius:
                mask[i,j] = (1 - (dist/radius) ** 2)

    result = cv2.merge([mask]*3) * img

    return result.astype(np.uint8)

def process(doc_img, back_img):
    preprocessed = preprocess_doc_image(doc_img)
    random_angle_1 = np.random.randint(-13, 13)
    random_angle_2 = np.random.randint(-13, 13)
    rotated_back = rotate_bound(back_img, random_angle_1)
    combined = put_image_on_background(preprocessed, rotated_back)

    projected = project_image(combined)
    rotated = rotate_bound(projected, random_angle_2)
    with_light = add_light(rotated)
    blurred_image = cv2.GaussianBlur(with_light, (9, 9), 0)

    return blurred_image



In [14]:
for i in range(10):
    doc_img = cv2.imread(os.path.join("data", "docs", f"{i + 1}.png"))
    print(f"doc={i + 1}")
    for j in range(10):
        print(f"\tback={j + 1}")
        back_img = cv2.imread(os.path.join("data", "background", f"{j + 1}.jpg"))
        result = process(doc_img, back_img)
        cv2.imwrite(f"result/doc{i + 1}_back{j + 1}.png", result)

doc=1
	back=1
	back=2
	back=3
	back=4
	back=5
	back=6
	back=7
	back=8
	back=9
	back=10
doc=2
	back=1
	back=2
	back=3
	back=4
	back=5
	back=6
	back=7
	back=8
	back=9
	back=10
doc=3
	back=1
	back=2
	back=3
	back=4
	back=5
	back=6
	back=7
	back=8
	back=9
	back=10
doc=4
	back=1
	back=2
	back=3
	back=4
	back=5
	back=6
	back=7
	back=8
	back=9
	back=10
doc=5
	back=1
	back=2
	back=3
	back=4
	back=5
	back=6
	back=7
	back=8
	back=9
	back=10
doc=6
	back=1
	back=2
	back=3
	back=4
	back=5
	back=6
	back=7
	back=8
	back=9
	back=10
doc=7
	back=1
	back=2
	back=3
	back=4
	back=5
	back=6
	back=7
	back=8
	back=9
	back=10
doc=8
	back=1
	back=2
	back=3
	back=4
	back=5
	back=6
	back=7
	back=8
	back=9
	back=10
doc=9
	back=1
	back=2
	back=3
	back=4
	back=5
	back=6
	back=7
	back=8
	back=9
	back=10
doc=10
	back=1
	back=2
	back=3
	back=4
	back=5
	back=6
	back=7
	back=8
	back=9
	back=10
